In [1]:
import utils as u
import torch
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from scipy.spatial import Delaunay
import trimesh
from scipy.spatial import distance_matrix


In [2]:
data = pd.read_csv('./dataset/anchor_points_dataset/data_points.csv')

In [3]:
# Reconstruct data

def reconstruct_data(data):
    data_tensor = torch.tensor(data[0:204])
    data_restructured = torch.zeros(size=(3,68)) 
    data_restructured[0, :] = data_tensor[0:68]
    data_restructured[1, :] = data_tensor[68:136]
    data_restructured[2, :] = data_tensor[136:204]
    return data_restructured

def calculate_centroid(data):
    sum_vectors = torch.sum(data, dim=0)
    centroid = sum_vectors / data.shape[0]
    return centroid

def calculate_gaussian_curvature(data):
    tri = Delaunay(data[:, :2])  
    faces = tri.simplices

    mesh = trimesh.Trimesh(vertices=data, faces=faces)
    gaussian_curvature = trimesh.curvature.discrete_gaussian_curvature_measure(mesh=mesh, points=data, radius=1)
    return gaussian_curvature

def calculate_pairwise_point_distances(data):
    return distance_matrix(data, data)

columns = [f'feature_{i}' for i in range(1, 205)] + ['label']
df_scaled_rotated = pd.DataFrame(columns=columns)

columns = [f'feature_{i}' for i in range(1, 202)] + ['label']
df_fft = pd.DataFrame(columns=columns)

columns = [f'feature_{i}' for i in range(1, 72)] + ['label']
df_gaussian = pd.DataFrame(columns=columns)

columns = [f'feature_{i}' for i in range(1, 4625)] + ['label']
df_pairwise = pd.DataFrame(columns=columns)

columns = [f'feature_{i}' for i in range(1, 4625 + 71)] + ['label']
df_shape_descriptors = pd.DataFrame(columns=columns)

for index, row in data.iterrows():
    
    row_list = row.values.flatten().tolist()
    label = row_list[-1]
    points = reconstruct_data(row_list)

    points = u.min_max_scale_data(u.rotate_face(points))


    # fft
    fft_vector = u.fft_feature_vector(points)
    

    # centroid
    centroid = calculate_centroid(points)

    # gaussian
    gaussian = calculate_gaussian_curvature(points)

    # pairwise
    pairwise = calculate_pairwise_point_distances(points)

    df_scaled_rotated.loc[index] = points.reshape(-1).tolist() + [label]
    df_fft.loc[index] = fft_vector.reshape(-1).tolist() + [label]
    df_gaussian.loc[index] = centroid.tolist() + gaussian.tolist() + [label]
    df_pairwise.loc[index] = pairwise.reshape(-1).tolist() + [label]
    df_shape_descriptors.loc[index] = centroid.tolist() + gaussian.tolist() + pairwise.reshape(-1).tolist() + [label]
    

    
df_scaled_rotated.to_csv('./dataset/anchor_points_dataset/' + 'rotated_data_points.csv', index=False)
df_fft.to_csv('./dataset/anchor_points_dataset/' + 'fft_data_points.csv', index=False)
df_gaussian.to_csv('./dataset/anchor_points_dataset/' + 'gaussian_data_points.csv', index=False)
df_pairwise.to_csv('./dataset/anchor_points_dataset/' + 'pairwise_data_points.csv', index=False)
df_shape_descriptors.to_csv('./dataset/anchor_points_dataset/' + 'shape_descriptors_data_points.csv', index=False)



